In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.firefox.options import Options

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd

In [2]:
def inicializa_selenium(gecko_path='./geckodriver'):
    """_summary_

    Args:
        gecko_path (str, optional): Path to geckodriver. Defaults to './geckodriver'.
    
    return: selenium.webdriver object
    """
    geckodriver_path = gecko_path
    options = Options()
    #options.add_argument('--headless')  # Se quiser rodar o Firefox em modo headless
    service = FirefoxService(executable_path=geckodriver_path)
    driver = webdriver.Firefox(service=service, options=options)
    return driver

def faz_login(usuario, senha, pagina_de_login='https://wop.obra.shalom.tec.br/login',driver=None):
    """_summary_

    Args:
        usuario (str, optional): Defaults to 'clerisson'.
        senha (str, optional): Defaults to 'clerissonshalom'.
        pagina_de_login (str, optional): efaults to 'https://wop.obra.shalom.tec.br/login'.
        driver (_type_, optional): objeto webdriver selenium a ser usado. Defaults to driver.
    """

    print("=> Fazendo Login")

    if not driver:
        driver = inicializa_selenium()

    
    driver.get(pagina_de_login)

    username_field = driver.find_element(By.NAME, 'username')  # Use o nome correto do campo de usuário
    password_field = driver.find_element(By.NAME, 'password')  # Use o nome correto do campo de senha

    username_field.clear()
    password_field.clear()
    username_field.send_keys(usuario)
    password_field.send_keys(senha)

    login_button = driver.find_element(By.XPATH, '//button[@type="submit"]')  # Use o seletor correto para o botão de login
    login_button.click()

    print("==> Ok")
    return 1

def pega_membros_do_grupo(id, driver=None,verbose=True):
    """Entra no link de um determinado grupo e cria um list de list contendo 
    as ovelhasjá cadastradas nesse grupo de oração.


    Args:
        id (_type_): id do grupo, str
        driver (_type_): selenium webdriver que vai acessar o link

    Returns:
        um list de lists no formato [[...], [...], [...]]
    """    

    print("=> Obtendo a lista de membros do grupo")

    id_do_grupo = id
    
    if not driver:
        try:
            driver = inicializa_selenium()
        except:
            print("Erro ao iniciar o Selenium")
            return "Erro"
    
    driver.get(f"https://wop.obra.shalom.tec.br/secured/local/grupodeoracao/{ id_do_grupo }/gerenciar")
    

    try:
        # Espere até que o elemento esteja visível (aguarde até 10 segundos)
        xpath= "/html/body/app-root/grupooracao-local-manage/secured/body/div/span/div/section[2]/div/div[4]/div/div[3]/div/div/table-component/div/div[2]/table/tbody"
        tabela = WebDriverWait(driver, 30).until(
            EC.visibility_of_element_located((By.XPATH, xpath))
        )
    except:
        print("=> O elemento não foi carregado a tempo")
        return None

    linhas = tabela.find_elements(By.TAG_NAME, 'tr')

    membros =[]
    for linha in linhas:
        colunas = linha.find_elements(By.TAG_NAME, 'td')
        dados = [coluna.text for coluna in colunas if coluna.text != '']
        membros.append(dados)

    print("==> Ok")
    return membros

In [5]:
# Inicializa Selenium e faz login no wop
driver = inicializa_selenium()

In [ ]:
# Obtém a lista dos grupos a partir do CSV
grupos_df = pd.read_csv("grupos.csv").sort_values(by=["CEV", "nome"])
grupos_df

In [ ]:
# Pega as informações de um determinado grupo
grupo_id = 300
path_to_export_csv = "./grupos/csv/"

membros_list= pega_membros_do_grupo(grupo_id, driver)

In [10]:
def gera_informações_do_grupo(id=301):
    id_do_grupo = id
    driver.get(f"https://wop.obra.shalom.tec.br/secured/local/grupodeoracao/{ id_do_grupo }/gerenciar")

    # Pega os pastores
    try:
        # Espere até que o elemento esteja visível (aguarde até 10 segundos)
        xpath= "/html/body/app-root/grupooracao-local-manage/secured/body/div/span/div/section[2]/div/div[2]/div/div[3]/div/div"
        pastores = WebDriverWait(driver, 30).until(
            EC.visibility_of_element_located((By.XPATH, xpath))
        )
    except:
        print("=> O elemento não foi carregado a tempo")
        return None
    
    linhas = tabela.find_elements(By.TAG_NAME, 'tr')

    print(linhas)
    
    membros =[]
    for linha in linhas:
        colunas = linha.find_elements(By.TAG_NAME, 'td')
        dados = [coluna.text for coluna in colunas if coluna.text != '']
        membros.append(dados)
    
    return membros
membros = gera_informações_do_grupo()
print(membros)

=> O elemento não foi carregado a tempo
None


In [ ]:
Grupo = pd.DataFrame(membros_list, columns=["user id","Nome", "Data de ingresso", "Status"])
Grupo = Grupo.sort_values(by='Nome')
Grupo

In [ ]:
Grupo.to_csv(path_to_export_csv,index=False)

In [ ]:
# Exporta todos os grupos
path_to_export_csv = "./grupos"
for grupo in grupos_df.itertuples():

    grupo_id = grupo.id
    grupo_nome = grupo.nome
    grupo_CEV = grupo.CEV
    grupo_nome = grupo_nome.replace(" ","_")

    texto  = f"{grupo_id}\t{grupo_nome}\t{grupo.CEV}"
    print(texto)

    membros = pega_membros_do_grupo(grupo_id,driver)

    if membros is None:
        print("=> Falhou")
    
    membros_df = pd.DataFrame(membros, columns=["user id","Nome", "Data de ingresso", "Status"])
    membros_df.sort_values(by='Nome')

    membros_df.to_csv(f"{path_to_export_csv}/{grupo_CEV}_{grupo_id}_{grupo_nome}.csv",index=False)


In [14]:
# Faechar o navegador
driver.quit()